## _Event Reader_

- event building (see using _`Event`_ class or _`Build_Event()`_ at _`src/event.py`_)
- event reading (see _`SttCSVDataReader, SttTorchDataReader`_ at _`src/reader.py`_)

In [ ]:
import glob, os, sys, yaml

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
# append parent dir
sys.path.append('..')

In [ ]:
# get cuda device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# local imports
from src import SttCSVDataReader, SttTorchDataReader
from src import detector_layout, detector_layout_new
from src import Build_Event, Build_Event_Viz
from src.math_utils import polar_to_cartesian

### _(1.) - Load Data_

In [ ]:
# path to the data (CSV)
input_dir = '../data_all'

### _(2.) - Event Building_

We have a single event in four different CSV files. One need to compose this event in single DataFrame.

In [ ]:
# select event
event_id = 1

#### _(2.1) - Event Class_

In [ ]:
# Test Event Class

#### _(2.2) - Build Event Function_

- _`Build_Event()` is same as `select_hits()` in `processing/utils/event_utils.py`_

In [ ]:
# build event
event = Build_Event(input_dir, event_id, noise=False, skewed=False, selection=False)

In [ ]:
# visualize event
Build_Event_Viz(event, figsize=(10,10), save_fig=False);

In [ ]:
event.head()

In [ ]:
# see column names
# event.keys().tolist()
# event.columns.tolist()

### _(3.) - Event Readers_

- test different data readers

#### _(3.1) - CSV Data Reader_

In [ ]:
# use SttCSVDataReader
inputdir = '../data_all'
csv_reader = SttCSVDataReader(inputdir,noise=False,skewed=False,selection=False)
data = csv_reader(evtid=event_id)

In [ ]:
# data.hits.head()
# data.tubes.head()
# data.particles.head()
# data.truth.head()
# data.event_file

In [ ]:
data.event.head()

- _Drop Duplicates in "Particles"_

In [ ]:
# particles = data.particles.copy() # copy() it otherwise event.particles is changed
# particles['nhits'] = particles.groupby(['particle_id'])['nhits'].transform('count')
# particles.drop_duplicates(inplace=True, ignore_index=True)
# particles.head(10)

- _Filter Particles in "Particles"_

In [ ]:
particles = data.particles.copy()

In [ ]:
particles.head()

In [ ]:
# there are particle we don't need e.g. electron (11), (1000010020), etc
particles.pdgcode.unique()

In [ ]:
# just keep protons, pions and maybe muon, don't forget resetting index and dropping old one.
particles[particles['pdgcode'].isin([-2212, 2212, -211, 211])].reset_index(drop=True)

- _renaming layers_

In [ ]:
# hits = data.hits.copy()

In [ ]:
# vlids = hits.layer_id.unique()
# n_det_layers = len(vlids)
# vlid_groups = hits.groupby(['layer_id'])
# hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer_id=i) for i in range(n_det_layers)]).reset_index(drop=True)

In [ ]:
# hits.head()

#### _(3.2) - Torch Data Reader_

In [ ]:
# use SttTorchDataReader
inputdir="../run/feature_store"
torch_reader = SttTorchDataReader(inputdir)
feature_data = torch_reader(evtid=event_id)

In [ ]:
feature_data

In [ ]:
# event id
event_id = int(os.path.basename(feature_data.event_file)[-4:])

In [ ]:
# node features (r, phi, z)
r  = feature_data.x[:, 0]
phi= feature_data.x[:, 1]
ir = feature_data.x[:, 2]

In [ ]:
r

In [ ]:
phi

In [ ]:
# vertex position
vx = feature_data.vertex[:, 0]
vy = feature_data.vertex[:, 1] 
vz = feature_data.vertex[:, 2]

In [ ]:
vx

In [ ]:
vy

In [ ]:
vz

In [ ]:
# get d0
d0 = torch.sqrt(vx**2+vy**2)

In [ ]:
d0

In [ ]:
# theta and phi angles
ptheta = feature_data.ptheta
pphi =  feature_data.pphi

In [ ]:
# radians to degrees
ptheta*180/np.pi

In [ ]:
# radians to degrees
pphi*180/np.pi

#### _Visualize Sample Events_